In [ ]:
from planet4 import markings
import hdbscan
plot_kwds = {'alpha' : 0.8, 's' : 80, 'linewidths':0}

In [ ]:
import seaborn as sns
%matplotlib inline
sns.set_context('poster')
sns.set_style('white')
sns.set_color_codes()

In [ ]:
id_ = 'b89'
id_ = '139'
id_ = 'dch'
id_ = 'bvc'
imid = markings.ImageID(id_, scope='planet4')

In [ ]:
blotches = imid.get_blotches()

In [ ]:
fans = imid.get_fans()

In [ ]:
imid.show_subframe();

In [ ]:
import matplotlib as mpl

In [ ]:
imid.plot_fans(with_center=True)

In [ ]:
data = fans
X = data['x y'.split()]
X = data['x y angle'.split()]

In [ ]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=4, gen_min_span_tree=True)

In [ ]:
clusterer.fit(X.as_matrix())

In [ ]:
fig, ax = plt.subplots(nrows=2)
# plt.figure()
imid.plot_blotches(img=True,ax=ax[0], with_center=True);
imid.plot_blotches(img=False,ax=ax[1], user_color='w');
clusterer.minimum_spanning_tree_.plot(axis=ax[1],edge_cmap='viridis', 
                                      edge_alpha=0.6, 
                                      node_size=80, 
                                      edge_linewidth=2);


In [ ]:
plt.figure()
clusterer.single_linkage_tree_.plot(cmap='viridis', colorbar=True)

In [ ]:
plt.figure()
clusterer.condensed_tree_.plot()

In [ ]:
plt.figure()
clusterer.condensed_tree_.plot(select_clusters=True, selection_palette=sns.color_palette())

In [ ]:
palette = sns.color_palette(n_colors=len(clusterer.labels_)-1)
cluster_colors = [sns.desaturate(palette[col], sat) 
                  if (col >= 0 and sat > 0.95) else (1.0, 1.0, 1.0) for col, sat in 
                  zip(clusterer.labels_, clusterer.probabilities_)]
test_data = X.as_matrix()
fig, ax = plt.subplots()
imid.show_subframe(ax=ax)
ax.scatter(test_data.T[0], test_data.T[1], c=cluster_colors, **plot_kwds)

In [ ]:
proba_cut=0.75

labels = clusterer.labels_
core_samples_mask = np.zeros_like(clusterer.labels_, dtype=bool)
core_samples_mask[clusterer.probabilities_ > proba_cut] = True
unique_labels = set(labels)
n_clusters = len(unique_labels) - (1 if -1 in labels else 0)
reduced_data = []  # list of `kind` cluster average objects
n_rejected = 0

# loop over unique labels.
for label in unique_labels:
    # get indices for members of this cluster
    class_member_mask = (labels == label)
    cluster_members = (class_member_mask & core_samples_mask)
    # treat noise
    if label == -1:
        n_rejected = len(cluster_members)
    # if label is a cluster member:
    else:
        reduced_data.append(cluster_members)


In [ ]:
from planet4 import markings
from scipy.stats import circmean

cols = markings.Fan.to_average
Marking = markings.Fan

data = fans

mean_markings = []
for cluster_members in reduced_data:
    clusterdata = data.loc[cluster_members, cols]
    meandata = clusterdata.mean()
    meandata.angle = np.rad2deg(
        circmean(
        np.deg2rad(
        clusterdata.angle)))
    cluster = Marking(meandata, scope='planet4')
    # storing n_members into the object for later.
    cluster.n_members = len(cluster_members)
    # storing this saved marker for later in ClusteringManager
    cluster.saved = False
    mean_markings.append(cluster)


In [ ]:
mean_markings